# Big Data w biznesie

## Trzeci notebook

In [ ]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """

   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles
   return HTML(s)
_set_css_style("../custom.css")

Pierwszym krokiem, jak zawsze, jest zaimportowanie potrzebnych nam bibliotek.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("./cars.csv")

# Dummy variables

Dummy variables to zmienne binarne, które są używane w analizie danych jako sposób na przekształcenie zmiennych kategorycznych na zmienne numeryczne, które mogą być używane w modelach statystycznych. Dummy variables tworzy się przez kodowanie każdej unikalnej wartości zmiennej kategorycznej jako nowej zmiennej binarnej, przyjmującej wartość 1, jeśli obserwacja należy do danej kategorii i 0 w przeciwnym przypadku.

Na przykład, jeśli mamy zmienną kategoryczną `"płeć"` z dwoma kategoriami `"kobieta"` i `"mężczyzna"`, to tworzymy dwie nowe zmienne binarne: `"kobieta"`, która przyjmuje wartość 1 dla kobiet i 0 dla mężczyzn oraz `"mężczyzna"`, która przyjmuje wartość 1 dla mężczyzn i 0 dla kobiet.

Dummy variables umożliwiają modelom statystycznym uchwycenie wpływu zmiennych kategorycznych na zmienną zależną w sposób numeryczny i jednoznaczny. Dummy variables są również ważne, gdy chcemy zbudować modele predykcyjne, ponieważ wiele algorytmów uczenia maszynowego nie akceptuje zmiennych kategorycznych w formie tekstowej i wymaga ich przekształcenia na wartości numeryczne.


Kolumna `'transmission'` przyjmuje tylko dwie wartości - `automatic` i `mechanical`.

In [ ]:
df['transmission']

Za pomocą funkcji `pd.get_dummies()` możemy zmienić dane kategoryczne na tzw. zmienne binarne (ang. binary variables) lub tzw. "dummy variables". Funkcja tworzy nową ramkę danych, w której dla każdej wartości kategorycznej z kolumny tworzy się nowa kolumna, która przyjmuje wartość 1 w przypadku, gdy dana wartość kategoryczna występuje w oryginalnej kolumnie, a 0 w przypadku, gdy jej tam nie ma.

In [ ]:
dummy_variable = pd.get_dummies(df['transmission'])
dummy_variable

Potem zmienimy nazwy kolumn.

In [ ]:
dummy_variable = dummy_variable.rename(columns={'mechanical':'transmission-mechanical', 'automatic':'transmission-automatic'})

Na końcu dołączymy stworzone "dummy_variable" do naszego DataFrame'a.

Często następnym krokiem byłoby usunięcie niepotrzebnej nam już kolumny `'transmission'`, ale w naszym przypadku przyda się jeszcze przy dalszej analizie.

In [ ]:
df = pd.concat([df, dummy_variable], axis=1)
#df = df.drop('transmission', axis = 1)
df

# Pivot tables

Pivot tables (także nazywane tabelami przestawnymi lub tabelami dynamicznymi) to narzędzie analizy danych, które umożliwia łatwe podsumowanie i grupowanie danych numerycznych z dużych zestawów danych w tabelach. Pivot tables pozwalają użytkownikowi na przekształcanie danych w sposób, który umożliwia łatwiejsze zrozumienie złożonych zależności między zmiennymi.

W pivot tables dane są ułożone w sposób przypominający tabelę, gdzie rzędy odpowiadają wartościom jednej zmiennej, a kolumny wartościom drugiej zmiennej. Pivot tables pozwalają na agregację danych w celu uzyskania sum, średnich, maksymalnych, minimalnych wartości, a także innych miar opisowych dla każdej kombinacji wartości zmiennych. Pivot tables pozwalają również na filtrowanie, sortowanie i grupowanie danych w sposób dynamiczny, co pozwala na uzyskanie bardziej szczegółowej analizy i zrozumienia złożonych zależności między zmiennymi.


Funkcja `pd.pivot_table()` w pandas pozwala użytkownikowi na tworzenie pivot tables. Najważniejsze argumenty tej funkcji to:

- data: obiekt DataFrame zawierający dane, które chcemy przetworzyć,
- index: kolumna lub lista kolumn, według których chcemy zgrupować dane,
- columns: kolumna lub lista kolumn, które chcemy przekształcić w nagłówki kolumn pivot table,
- values: kolumna lub lista kolumn, z których chcemy wyliczyć statystyki,
- aggfunc: funkcja lub lista funkcji agregujących, które chcemy zastosować do kolumn values.

Pivot tables umożliwiają również wykorzystanie wielu funkcji agregujących, takich jak sum, mean, count, min, max, itd. Ostatecznie, wynik pivot table może być sortowany i filtrowany w taki sam sposób jak standardowy DataFrame.

Pivot tables są szczególnie przydatne do analizy dużych zbiorów danych i ułatwiają wyciąganie z nich wniosków i ukrytej wiedzy.

Na przykład chcemy zobaczyć, jakie są średnie wartości cen samochodów w zależności od typu nadwozia i rodzaju napędu. Możemy to zrobić w następujący sposób:

In [ ]:
pivot_table = pd.pivot_table(df, values='price_usd', index='drivetrain', columns='body_type', aggfunc='mean')
pivot_table

W przypadku, gdy w pivot table znajduje się wartość NaN, można zastąpić ją inną wartością, np. wartością średnią dla danego wiersza lub kolumny, lub wartością 0. Można to zrobić za pomocą funkcji `df.fillna()` w bibliotece pandas. Przykładowo, można zastąpić wszystkie wartości NaN wartością średnią dla danej kolumny, używając kodu:


In [ ]:
pivot_table = pivot_table.fillna(pivot_table.mean(axis=0))
pivot_table

Tak wygląda kod, jeśli chcemy zastąpić wszystkie brakujące wartości zerami.

In [ ]:
pivot_table = pivot_table.fillna(0)
pivot_table

Jeśli chodzi o zastąpienie brakujących wartości średnią wartością z rzędu, należy wziąć pod uwagę, że metoda df.fillna(df.mean(axis=1)) (z argumentem axis=1, oznaczającym obliczenie średniej z całego rzędu) nie jest dostępna w bibliotece pandas. Można jednak obejść ten problem stosując następujący kod.

In [ ]:
pivot_table = pivot_table.apply(lambda row: row.fillna(row.mean()), axis=1)
pivot_table

***
Jest też inny sposób na stworzenie pivot table bez użycia funkcji `pd.pivot_table()`.

Możemy najpierw stworzyć DataFrame zawierający wartości średnie ceny w zależności od połączenia wartości kolumn `'drivetrain'` i `'body_type'`.

In [ ]:
df_pivot = df.groupby(['drivetrain', 'body_type'], as_index=False)['price_usd'].mean()
df_pivot

A następnie za pomocą funkcji `df.pivot()` przekształcić w tabelę przestawną.

In [ ]:
df_pivot_table = df_pivot.pivot(index='drivetrain', columns='body_type')
df_pivot_table

Kombinacja obu funkcji może być alternatywą dla `pd.pivot_table()` w zależności od potrzeb.

Jednak `pd.pivot_table()` pozwala na bardziej elastyczne definiowanie operacji agregujących, kształt wynikowej tabeli przestawnej, a także sposobu traktowania brakujących danych.

# Stylowanie

Styling w pandas to proces formatowania wyjścia DataFrame lub Series w celu poprawienia czytelności i estetyki prezentowanych danych. Funkcje stylingowe umożliwiają dostosowywanie wyjścia w zależności od wartości komórek, np. zmiany koloru lub formatowania tekstu. Za pomocą stylizacji można także łączyć wartości komórek w jednej kolumnie, dodawać tła czy łączyć komórki w obramowania.

Metoda `df.style()` w bibliotece Pandas służy do formatowania i stylizacji tabel. Opcje, które można zastosować w tej metodzie to między innymi:

- background_gradient(): wypełnienie komórek gradientem kolorów w zależności od wartości,
- highlight_max() / highlight_min(): wyróżnienie największych / najmniejszych wartości w tabeli,
- format(): formatowanie wartości w komórkach,
- set_caption(): pozwala na ustawienie tytułu dla tabeli,
- bar(): wyświetla słupki dla każdej komórki w kolumnie, na podstawie jej wartości,
- hide_index(): ukrycie indeksu tabeli,
- hide_columns(): ukrycie wybranych kolumn,
- set_properties(): ustawianie różnych właściwości CSS dla komórek m.in.:
    - background-color - kolor tła
    - color - kolor tekstu
    - font-weight - waga czcionki
    - text-align - wyrównanie tekstu
    - border - styl obramowania
    - font-size - rozmiar czcionki
    - opacity - przejrzystość elementu

Oprócz tego istnieje wiele innych opcji, które pozwalają na dostosowanie wyglądu tabeli do konkretnych potrzeb.

Możemy stworzyć i wyświetlić tabele z losowymi danymi.

In [ ]:
df_rand = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])
df_rand

Zmodyfikujemy kilka parametrów, aby dostosować wygląd naszej tabeli.

In [ ]:
df_rand.style\
    .format('{:.3f}', na_rep="")\
    .background_gradient(cmap="Blues")\
    .set_caption("Tabela 1. Przypadkowe dane")\
    .highlight_min()\
    .highlight_max()

Jeśli stylowanie tabelki nie odpowiada naszym oczekiwaniom, istnieje wiele sposobów na dostosowanie jej wyglądu.

In [ ]:
df_rand.style\
   .format('{:.3f}', na_rep="")\
   .background_gradient(cmap="Blues")\
   .hide(axis="index")\
   .set_caption("Tabela 1. Przypadkowe dane")\
   .set_table_styles([{'selector': 'th',
                       'props': [('font-weight', 'bold'),
                                 ('text-align', 'center'),
                                 ('font-size', '13pt')]},
                      {'selector': 'caption',
                       'props': [('font-weight', 'bold'),
                                 ('color', 'white'),
                                 ('font-size', '16pt')]}])\
   .highlight_min(color="Red")\
   .highlight_max(color="Green")\
   .set_properties(padding='8px 20px')

Używanie stylowania w pandas ma wiele zalet w porównaniu do korzystania z zewnętrznego programu. Kilka z tych zalet to:

- Łatwość i wygoda: Stylowanie w pandas jest bardzo łatwe i wygodne, ponieważ pozwala na szybkie i łatwe dostosowanie wyglądu tabeli bez konieczności korzystania z zewnętrznych programów.

- Powtarzalność: Zastosowanie stylów w pandas pozwala na zastosowanie tego samego stylu w różnych miejscach tabeli lub nawet w różnych tabelach, co zapewnia spójność i łatwość w utrzymaniu.

- Integracja: Korzystanie z stylowania w pandas pozwala na łatwe zintegrowanie stylu tabeli z innymi funkcjonalnościami pandas, takimi jak grupowanie danych, sortowanie, filtrowanie itp.

- Oszczędność czasu: Korzystanie z stylowania w pandas pozwala zaoszczędzić czas, ponieważ nie trzeba eksportować danych do zewnętrznego programu, stosować tam stylów, a następnie importować zmienione dane z powrotem do pandas.

- Elastyczność: Stylowanie w pandas zapewnia dużą elastyczność w dostosowaniu wyglądu tabeli do konkretnych potrzeb, co pozwala na uzyskanie bardziej dokładnego i spójnego wyglądu w porównaniu z zewnętrznymi programami.

# Wizualizacje

Umiejętność wizualizacji jest ważna w wielu dziedzinach, takich jak nauki społeczne, nauki przyrodnicze, biznes, finanse, medycyna itp. Poprzez wizualizacje możemy prezentować dane w sposób, który jest łatwiejszy do zrozumienia i może pomóc w identyfikowaniu wzorców i zależności. Wizualizacje mogą również pomóc w wyjaśnianiu trudnych koncepcji i prezentowaniu wyników badań w sposób przystępny dla różnych odbiorców.

Najpopularniejszymi bibliotekami do wizualizacji w Pythonie są:

- Matplotlib - biblioteka pozwalająca na tworzenie różnego rodzaju wykresów, histogramów i innych wizualizacji danych w Pythonie.

- Seaborn - biblioteka wykorzystująca Matplotlib, ale oferująca bardziej zaawansowane narzędzia do wizualizacji danych, w tym specjalistyczne wykresy do analizy statystycznej.

- Plotly - biblioteka umożliwiająca tworzenie interaktywnych wykresów, map i innych wizualizacji danych w Pythonie.

- Bokeh - biblioteka umożliwiająca tworzenie dynamicznych wizualizacji danych, w tym interaktywnych wykresów, map i dashboardów.

- ggplot - biblioteka wzorowana na popularnym pakiecie ggplot2 w języku R, pozwalająca na tworzenie wizualizacji danych zgodnych z zasadami "Grammar of Graphics".

Oprócz tych bibliotek istnieje wiele innych narzędzi do wizualizacji danych w Pythonie, w tym Altair, HoloViews, geopandas, czy Choropleth.

Na zajęciach będziemy głównie zajmować się tylko najpopularniejszymi bibliotekami, czyli - `matplolib` i `seaborn`. Zanim jednak do tego przejdziemy, warto poznać możliwości innej prostszej bibliteki.

# Pandas - wizualizacja

Biblioteka Pandas, oprócz umożliwienia łatwej i wydajnej obróbki danych, oferuje także wiele wbudowanych funkcji wizualizacji danych. Dzięki temu, że wizualizacja jest zintegrowana z danymi, jest to bardzo wygodne narzędzie do eksploracji i analizy danych. Do najpopularniejszych metod wizualizacji danych w Pandas należą wykresy liniowe, wykresy słupkowe, histogramy, wykresy kołowe, wykresy punktowe, wykresy pudełkowe i wiele innych. Dzięki temu, że funkcje wizualizacji Pandas działają na ramkach danych, można łatwo manipulować danymi, filtrować je i sortować, a następnie wizualizować wyniki, aby lepiej zrozumieć zależności między różnymi zmiennymi w danych. Wizualizacja danych jest niezwykle ważna dla zrozumienia i prezentowania wyników, więc umiejętność korzystania z narzędzi wizualizacji danych w Pandas może być bardzo przydatna dla analityków danych i naukowców danych.

Oto niektóre narzędzia do wizualizacji dostępne w bibliotece pandas:

- df.plot() - umożliwia tworzenie różnych rodzajów wykresów, w tym liniowych, punktowych, słupkowych, histogramów i wiele innych.
- df.hist() - służy do tworzenia histogramów.
- df.boxplot() - tworzy wykres pudełkowy.
- df.scatter() - służy do tworzenia wykresów punktowych.
- df.area() - tworzy wykres obszaru.
- df.bar() - tworzy wykres słupkowy.
- df.pie() - tworzy wykres kołowy.
- df.plot.hexbin() - służy do tworzenia wykresów heksagonalnych.
- df.plot.kde() - służy do tworzenia wykresów gęstości jądrowych.
- df.plot.density() - tworzy wykres gęstości.
- df.plot.scatter() - tworzy wykres punktowy z dwoma zmiennymi.
- df.plot.barh() - tworzy wykres słupkowy poziomy.
- df.plot.hist() - służy do tworzenia histogramów.
- df.plot.box() - tworzy wykres pudełkowy.
- df.plot.area() - tworzy wykres obszaru.

Przy pomocy pandas chcemy wygenerować wykres punktowy typu scatter plot, który przedstawia zależność między dwiema zmiennymi: `'year_produced'` (rok produkcji) na osi x i `'odometer_value'` (wartość przebiegu) na osi y.

In [ ]:
df.plot(kind='scatter',
        x='year_produced',
        y='odometer_value',
        title='Year vs Odometer')

Parametry funkcji do wizualizacji w bibliotece pandas mogą się różnić w zależności od konkretnej funkcji, ale często występujące parametry to:

- x - kolumna DataFrame do umieszczenia na osi x
- y - kolumna DataFrame do umieszczenia na osi y
- kind - rodzaj wykresu, np. 'line' dla wykresu liniowego, 'bar' dla wykresu słupkowego
- color - kolor lub paleta kolorów do zastosowania w wykresie
- title - tytuł wykresu
- xlabel - etykieta osi x
- ylabel - etykieta osi y
- legend - określa, czy wyświetlić legendę, zazwyczaj jako wartość True lub False
- figsize - wymiary figury, np. (10, 6) oznacza szerokość 10 i wysokość 6 cali
- xlim - zakres osi x
- ylim - zakres osi y
- bins - liczba koszyków (słupków) dla wykresu histogramu
- alpha - wartość przezroczystości, z zakresu od 0 (przezroczysty) do 1 (nieprzezroczysty)
- linewidth - grubość linii w wykresie
- edgecolor - kolor krawędzi w wykresie
- grid - określa, czy włączyć siatkę, zazwyczaj jako wartość True lub False

Jednakże, pełna lista parametrów znajduje się w dokumentacji biblioteki pandas i może się różnić w zależności od konkretnej funkcji.

Biblioteka pandas ma kilka funkcji do tworzenia prostych wykresów, które są szybkie i proste do zastosowania. Jednakże, biblioteki matplotlib i seaborn oferują o wiele większą elastyczność i bardziej zaawansowane funkcje do tworzenia wizualizacji danych. Dzięki temu możemy tworzyć bardziej skomplikowane wykresy, dostosowywać ich wygląd do własnych potrzeb, dodawać opisy, tytuły, legendy, a także stosować różne style i kolory. Dodatkowo biblioteki te oferują wiele zaawansowanych narzędzi statystycznych do analizy danych, które pozwalają na bardziej zaawansowaną analizę danych. W związku z tym, jeśli potrzebujemy bardziej skomplikowanej wizualizacji lub analizy danych, warto skorzystać z bibliotek matplotlib lub seaborn, zamiast korzystać tylko z funkcji wizualizacji w bibliotece pandas.

# Matplotlib - wizualizacja

Matplotlib to biblioteka w języku Python służąca do tworzenia wykresów i wizualizacji danych. Jest to jedna z najpopularniejszych bibliotek do wizualizacji w Pythonie. Matplotlib oferuje wiele narzędzi do rysowania różnych rodzajów wykresów, takich jak wykresy liniowe, wykresy punktowe, wykresy kołowe, wykresy słupkowe, histogramy, wykresy 3D i wiele innych. Biblioteka ta oferuje również wiele opcji dostosowywania wykresów, aby uzyskać pożądany wygląd i formatowanie.

Przed rozpoczęciem korzystania z biblioteki musimy ją zaimportować.

Pierwsza linijka import matplotlib.pyplot as plt importuje moduł pyplot z biblioteki matplotlib i nadaje mu alias plt, co pozwala na wywoływanie funkcji z tego modułu za pomocą skrótu plt zamiast pełnej nazwy matplotlib.pyplot. Moduł pyplot zawiera funkcje do tworzenia wykresów i wizualizacji danych.

Druga linijka %matplotlib inline jest tzw. magiczną komendą IPythona, która ustawia backend inline dla biblioteki matplotlib. Backend określa sposób, w jaki matplotlib renderuje wykresy, a wartość inline oznacza, że wykresy będą renderowane bezpośrednio w jupyter notebooku pod kodem, a nie w oddzielnym oknie aplikacji.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Na początek możemy spróbować stworzyć taki sam wykres punktowy, który przedstawia zależność między dwiema zmiennymi: `'year_produced'` (rok produkcji) na osi x i `'odometer_value'` (wartość przebiegu) na osi y.

In [ ]:
plt.scatter(df['year_produced'], df['odometer_value'])
plt.xlabel("year_produced")
plt.ylabel("odometer_value")
plt.title('Year vs Odometer')

Oto lista kilku narzędzi do wizualizacji w bibliotece matplotlib:

- Wykresy liniowe - plot(), scatter(), stem(), step(), fill()
- Wykresy słupkowe - bar(), barh()
- Wykresy kołowe - pie()
- Wykresy histogramów - hist()
- Wykresy pudełkowe - boxplot()
- Wykresy punktowe - scatter(), stripplot(), swarmplot()
- Wykresy powierzchniowe - contour(), contourf(), imshow(), pcolor(), pcolormesh(), surface()
- Wykresy 3D - plot_surface(), scatter(), plot_wireframe()
- Wykresy czasowe - plot_date(), bar()
- Wykresy interaktywne - plotly(), bokeh(), mpld3()

Możemy spróbować zwizualizować pivot table, którą stworzyliśmy wcześniej.

In [ ]:
pivot_table

Funkcja `plt.pcolor()` służy do wyświetlania macierzy kolorów w zależności od wartości danych numerycznych. Przyjmuje jako argumenty macierz, która ma być wyświetlona i opcjonalnie mapę kolorów (w tym wypadku 'RdBu' - oznaczająca kolor czerwono-niebieski).

Funkcja `plt.colorbar()` wyświetla legendę mapy kolorów, która odpowiada wartościom numerycznym.

Funkcja `plt.show()` wyświetla wykres z zastosowanymi funkcjami i w tym przypadku przedstawia pivot table jako macierz kolorów z legendą.

In [ ]:
plt.pcolor(pivot_table, cmap='seismic')
plt.colorbar()
plt.show()

Biblioteka Matplotlib oferuje wiele funkcji i parametrów do tworzenia wizualizacji. Poniżej wymieniam kilka z nich:

- figure(): tworzy nowy obiekt figury, umożliwiając wyświetlenie i zapisanie wykresu.
- add_subplot(): tworzy nowy subplot na obrazie.
- plot(): tworzy liniowy wykres, wykres punktowy lub histogram.
- scatter(): tworzy wykres punktowy z punktami rozproszonymi na płaszczyźnie.
- bar(): tworzy wykres słupkowy lub wykres kolumnowy.
- hist(): tworzy histogram.
- boxplot(): tworzy wykres pudełkowy.
- set_xlabel(): ustawia etykietę dla osi X.
- set_ylabel(): ustawia etykietę dla osi Y.
- set_title(): ustawia tytuł wykresu.
- legend(): dodaje legendę do wykresu.
- xlim(): ustawia zakres dla osi X.
- ylim(): ustawia zakres dla osi Y.
- subplots_adjust(): dostosowuje rozmiar subplotów na obrazie.

Są to tylko niektóre z wielu dostępnych funkcji i parametrów.

Chcemy zmienić wygląd wygenerowanej przez pivot_table tabeli na bardziej czytelną i estetyczną formę. Dokonamy kilku modyfikacji, takich jak zmiana kolorów i wyrównanie etykiet, aby tabela była łatwiejsza do odczytania. Dodamy również pionową skalę kolorów, która odpowiada wartościom numerycznym w tabeli.

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolor(pivot_table, cmap='seismic')

#label names
row_labels = pivot_table.columns
col_labels = pivot_table.index

#move ticks and labels to the center
ax.set_xticks(np.arange(pivot_table.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(pivot_table.shape[0]) + 0.5, minor=False)

#insert labels
ax.set_xticklabels(row_labels, minor=False)
ax.set_yticklabels(col_labels, minor=False)

#rotate label if too long
plt.xticks(rotation=90)

fig.colorbar(im)
plt.show()

# Binning

Wykonamy jeszcze kilka wykresów za pomocą matplotliba, ale najpierw dowiemy się co to jest binning.

Binning to technika przekształcania danych ciągłych na kategorie, czyli zakresy wartości. Polega na podziale wartości zmiennej ciągłej na określoną liczbę przedziałów (binów) i przypisaniu każdej wartości do odpowiedniego przedziału. Binning może pomóc w analizie danych, szczególnie gdy dane są bardzo skomplikowane lub zawierają dużo szumu.

Binning służy do grupowania danych w określone przedziały (np. zakresy wartości) zwane "koszami" (ang. bins), co umożliwia analizę danych w bardziej ustrukturyzowany sposób. Po zastosowaniu binningu można wykonać szereg operacji analitycznych, takich jak wyznaczenie średniej, sumy, odchylenia standardowego itp. dla każdego kosza. Binning może również pomóc w zredukowaniu szumu i zwiększeniu wydajności obliczeniowej, ponieważ zmniejsza liczbę unikalnych wartości danych do przetworzenia.

Przyjmijmy, że chcemy dokonać analizy ilości samochodów według ich przebiegu, ale nie jest dla nas istotna dokładna wartość, a jedynie przynależność do jednej z trzech kategorii: "Niski", "Średni" i "Wysoki".

Najpierw znajdziemy wartości graniczne między naszymi kategoriami przy pomocy takiego kodu:

In [ ]:
bins = np.linspace(min(df['odometer_value']), max(df['odometer_value']), 4)
bins

Dokładnie do tego posłuży nam funkcja `pd.cut()`. Funkcja ta tworzy kategorie na podstawie podanego zestawu przedziałów (bins), do których przypisywane są wartości z analizowanego zbioru danych. W wyniku działania funkcji `pd.cut()` powstaje kategoryczna zmienna, której wartości określają, do której kategorii przypisane są wartości z analizowanego zbioru danych.

In [ ]:
group_names = ['Low', 'Medium', 'High']
df['odometer_value_binned'] = pd.cut(df['odometer_value'], bins, labels=group_names, include_lowest=True )
df[['odometer_value','odometer_value_binned']]

Możemy zwizualizować nasze kosze (biny) za pomocą histogramu. Wykorzystamy bibliotekę matplotlib, która pozwala na łatwe generowanie histogramów, a kod wygląda następująco:

In [ ]:
plt.hist(df['odometer_value_binned'], alpha=0.5, edgecolor='black', linewidth=1.2)
plt.xlabel("Odometer Value")
plt.ylabel("Count")
plt.title("Distribution of Odometer Values")
plt.show()

To samo możemy uzyskać, używając parametru `'bins'` podczas tworzenia histogramu, bez konieczności tworzenia osobnych koszy (binów) na podstawie danych.

In [ ]:
plt.hist(df['odometer_value'], bins=3)
plt.xlabel("Odometer Value")
plt.ylabel("Count")
plt.title("Distribution of Odometer Values")
bin_centers = 0.5 * (bins[:-1] + bins[1:])
plt.xticks(bin_centers, group_names)
plt.show()

# Seaborn - wizualizacja

 Seaborn to biblioteka w języku Python do wizualizacji danych oparta na matplotlib. Jest między nimi kilka różnic:

- Poziom skomplikowania: Matplotlib jest bardziej podstawową biblioteką i oferuje większą elastyczność w tworzeniu własnych wykresów. Seaborn natomiast oferuje bardziej zaawansowane funkcje i domyślne ustawienia, co umożliwia szybkie tworzenie estetycznych i profesjonalnych wykresów.

- Styl: Seaborn oferuje wbudowane style, które są bardziej atrakcyjne wizualnie niż domyślne style w Matplotlib. Style te oferują większą różnorodność palet kolorów, poprawione czcionki i linie siatki.

- Domyślne ustawienia: Seaborn oferuje bardziej precyzyjne domyślne ustawienia osi i skali, co ułatwia tworzenie estetycznych wykresów. Matplotlib ma bardziej elastyczne ustawienia, co oznacza, że użytkownik musi je dostosować samodzielnie.

- Typy wykresów: Seaborn oferuje kilka unikalnych typów wykresów, takich jak wykresy wstęgowe, wykresy punktowe z dopasowaną linią regresji czy wykresy violina. Matplotlib oferuje bardziej podstawowe typy wykresów, takie jak wykresy liniowe, słupkowe czy punktowe.

- Złożoność kodu: Ze względu na bardziej precyzyjne domyślne ustawienia, Seaborn wymaga mniejszej ilości kodu niż Matplotlib do osiągnięcia podobnych wyników.

Ogólnie rzecz biorąc, Seaborn oferuje łatwiejszy w użyciu interfejs oraz bardziej zaawansowane funkcje do tworzenia estetycznych wykresów, podczas gdy Matplotlib oferuje większą elastyczność w tworzeniu niestandardowych wykresów. Wiele osób decyduje się na użycie Seaborn jako ulepszonej wersji Matplotlib.

Z tego powodu przez resztę zajęć zazwyczaj będziemy korzystać z Seaborna.



Jak zawsze pierwszym krokiem jest zaimportowanie biblioteki.

In [ ]:
import seaborn as sns

Na początek możemy spróbować stworzyć dokładnie taki sam wykres punktowy jak poprzednio, który przedstawia zależność między rokiem produkcji, a wartością przebiegu.

In [ ]:
sns.scatterplot(x="year_produced", y="odometer_value", data=df)
plt.show()

Oto niektóre narzędzia do wizualizacji w bibliotece seaborn:

- lineplot(): tworzy wykres liniowy
- scatterplot(): tworzy wykres punktowy
- barplot(): tworzy wykres słupkowy
- histplot(): tworzy histogram
- boxplot(): tworzy wykres pudełkowy
- heatmap(): tworzy mapę ciepła
- pairplot(): tworzy wykresy punktowe dla wszystkich kombinacji zmiennych
- regplot(): tworzy wykres punktowy z dopasowaną linią regresji
- jointplot(): wykres zależności między dwoma zmiennymi wraz z histogramami każdej zmiennej
- violinplot(): wykres skrzypcowy przedstawiający rozkład danych dla różnych kategorii
- swarmplot(): wykres punktowy przedstawiający rozkład danych dla różnych kategorii, często używany razem z violinplot()
- factorplot(): ogólny wykres kategorii, który umożliwia rysowanie różnych rodzajów wykresów dla różnych typów danych
- countplot(): wykres słupkowy przedstawiający liczbę wystąpień dla każdej wartości w kategorii
- lmplot(): wykres zależności między dwoma zmiennymi wraz z linią regresji
- catplot(): ogólny wykres kategorii, który umożliwia rysowanie różnych rodzajów wykresów dla różnych typów danych, w tym także dodawanie subplots.

Teraz, korzystając z naszych danych, pokażemy jak narysować większość typów wykresów.

In [ ]:
# Tworzenie wykresu liniowego
sns.lineplot(x='year_produced', y='price_usd', data=df)
plt.ylim(0,)
plt.show()

In [ ]:
# Tworzenie wykresu punktowego
sns.scatterplot(x='engine_capacity', y='price_usd', data=df)
plt.xlim(0,)
plt.ylim(0,)
plt.show()

In [ ]:
# Tworzenie wykresu słupkowego
sns.barplot(x='body_type', y='price_usd', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Tworzenie histogramu
sns.histplot(x='odometer_value', data=df)
plt.xlim(0,)
plt.show()

In [ ]:
# Tworzenie wykresu pudełkowego
sns.boxplot(x='transmission', y='price_usd', data=df)
plt.ylim(0,)
plt.show()

In [ ]:
# Tworzenie mapy ciepła
sns.heatmap(pivot_table, cmap='seismic')
plt.show()

In [ ]:
# Tworzenie wykresów punktowych dla wszystkich kombinacji zmiennych
sns.pairplot(df[['odometer_value', 'engine_capacity', 'price_usd', 'year_produced']])
plt.show()

In [ ]:
# Tworzenie wykresu punktowego z dopasowaną linią regresji
sns.regplot(x='odometer_value', y='price_usd', data=df, scatter_kws={'s': 2}, line_kws={'color': 'red', 'lw': 2})
plt.xlim(0,)
plt.ylim(0,)
plt.show()

In [ ]:
# wykres zależności między dwoma zmiennymi wraz z histogramami każdej zmiennej
sns.jointplot(x='year_produced', y='price_usd', data=df)
plt.ylim(0,)
plt.show()

In [ ]:
# wykres skrzypcowy przedstawiający rozkład danych dla różnych kategorii
sns.violinplot(x='manufacturer_name', y='price_usd', data=df, color='yellow', linewidth=0.1)
plt.ylim(0,)
plt.xticks(rotation=90)
plt.tick_params(axis='x', labelsize=8)
plt.show()

In [ ]:
# wykres słupkowy przedstawiający liczbę wystąpień dla każdej wartości w kategorii
sns.countplot(x='engine_fuel', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# wykres zależności między dwoma zmiennymi wraz z linią regresji
sns.lmplot(x='year_produced', y='price_usd', data=df)
plt.ylim(0,)
plt.show()

In [ ]:
# ogólny wykres kategorii, który umożliwia rysowanie różnych rodzajów wykresów dla różnych typów danych, w tym także dodawanie subplots
sns.catplot(x='engine_type', y='price_usd', hue='transmission', kind='violin', col='drivetrain', data=df, palette='seismic')
plt.ylim(0,)
plt.show()

W bibliotece Seaborn, parametry funkcji wizualizacyjnych różnią się w zależności od konkretnej funkcji. Niemniej jednak, istnieją pewne często spotykane parametry, które możemy wyróżnić, takie jak:

- data: DataFrame lub inny obiekt, który zawiera dane do wizualizacji.
- x, y: nazwy kolumn lub indeksy, które określają zmienne, które chcemy porównać lub wizualizować.
- hue: nazwa kolumny lub indeksu, która umożliwia dodatkowe porównywanie danych w oparciu o inną zmienną.
- palette: nazwa palety kolorów lub lista kolorów, która określa schemat kolorów do wykorzystania w wizualizacji.
- order, hue_order: listy wartości, które określają porządek wyświetlania danych na wykresie.
- kind: określa rodzaj wykresu do narysowania, na przykład 'bar', 'box', 'scatter', itp.
- size, aspect: kontrolują wielkość i proporcje rysowanego wykresu.
- title, xlabel, ylabel: napisy, które określają tytuł i etykiety osi wizualizacji.

Oczywiście to tylko kilka przykładów, a w zależności od funkcji, można spotkać różne parametry związane z ustawieniem stylu, dodawaniem elementów takich jak legendy, siatki, etykiety punktów i wiele innych.

# Korelacja

Korelacja to miara, która określa, jak silnie dwie zmienne są ze sobą związane. Przyjmuje wartości od -1 do 1, gdzie wartości bliskie -1 oznaczają silną ujemną korelację, wartości bliskie 1 oznaczają silną dodatnią korelację, a wartość 0 oznacza brak korelacji.

W pandasie korelację można badać za pomocą metody `df.corr()`. Domyślnie metoda ta zwraca tabelę korelacji dla wszystkich kolumn z wartościami numerycznymi w danym obiekcie DataFrame. Jednak w przyszłych wersjach, wartość domyślna argumentu numeric_only zostanie zmieniona na False. Można również podać argument `method`, który określa metodę obliczania korelacji, np. `pearson` (domyślna), `kendall` lub `spearman`.

In [ ]:
df.corr(numeric_only=True)

Możemy zwizualizować korelację pomiędzy różnymi kolumnami w obiekcie DataFrame za pomocą heatmapy. Wizualizacja ta pozwala nam na szybkie zidentyfikowanie silnych pozytywnych i negatywnych korelacji między zmiennymi.

In [ ]:
sns.heatmap(df.corr(numeric_only=True), cmap='gray')
plt.show()

Zbadajmy trzy możliwości:
- korelacja dodatnia
- korelacja bliska zeru
- korelacja ujemna

Na początek sprawdźmy korelacje pomiędzy rokiem produkcji a ceną samochodu.

In [ ]:
df[["year_produced", "price_usd"]].corr()

Jak widzimy, korelacja między dwiema zmiennymi jest dodatnia, co w praktyce oznacza, że kiedy jedna zmienna rośnie, to druga również rośnie.

Sprawdźmy teraz, czy ten sam trend będzie widoczny na wykresie z regresją liniową.

In [ ]:
sns.regplot(x="year_produced", y="price_usd", data=df, scatter_kws={'s': 2}, line_kws={'color': 'red', 'lw': 2})
plt.xlim(df["year_produced"].min(), df["year_produced"].max())
plt.ylim(0, )
plt.show()

Wraz ze wzrostem roku produkcji można zaobserwować wzrost ceny, co sugeruje istnienie dodatniej bezpośredniej korelacji między tymi dwoma zmiennymi.

Następnie sprawdźmy korelacje pomiędzy rokiem produkcji a wielkością silnika.

In [ ]:
df[["year_produced", "engine_capacity"]].corr()

Korelacja bliska zeru sugeruje, że między tymi dwoma zmiennymi nie ma silnego związku.

Sprawdźmy teraz, czy potwierdzi się to na wykresie z regresją liniową.

In [ ]:
sns.regplot(x="year_produced", y="engine_capacity", data=df, scatter_kws={'s': 2}, line_kws={'color': 'red', 'lw': 2})
plt.xlim(df["year_produced"].min(), df["year_produced"].max())
plt.ylim(0, )
plt.show()

Linia regresji jest prawie pozioma. Ponadto punkty są bardzo rozproszone i oddalone od dopasowanej linii, co pokazuje dużą zmienność. Może to sugerować brak związku między tymi dwoma zmiennymi.

Na koniec sprawdźmy korelacje pomiędzy rokiem produkcji a przebiegiem.

In [ ]:
df[["year_produced", "odometer_value"]].corr()

Jak możemy zauważyć, między tymi dwiema zmiennymi występuje ujemna korelacja, co oznacza, że gdy wartość jednej zmiennej wzrasta, to wartość drugiej zmiennej maleje.

Sprawdźmy teraz, czy ten sam trend będzie widoczny na wykresie z regresją liniową.

In [ ]:
sns.regplot(x="year_produced", y="odometer_value", data=df, scatter_kws={'s': 2}, line_kws={'color': 'red', 'lw': 2})
plt.xlim(df["year_produced"].min(), df["year_produced"].max())
plt.ylim(0, )
plt.show()

Wraz ze wzrostem roku produkcji przebieg maleje: to wskazuje na odwrotny ujemny związek między tymi dwoma zmiennymi.


Na koniec musimy pamiętać, że korelacja nie wskazuje na związek przyczynowo-skutkowy między zmiennymi, a jedynie na to, jak bardzo są one ze sobą związane liniowo. Wysoka lub niska wartość korelacji nie oznacza bezpośrednio przyczynowości, lub jej braku, ponieważ istnieć mogą inne czynniki, które wpływają na obie zmienne. Niska wartość korelacji również nie oznacza wprost braku związku między zmiennymi, ponieważ może istnieć nieliniowa zależność między nimi, która nie zostanie ujawniona przez korelację liniową.

W przyszłości wielkość korelacji między dwoma zmiennymi może nam sugerować, czy jedna zmienna może być dobrym predyktorem drugiej zmiennej. Im silniejsza korelacja między zmiennymi, tym większe prawdopodobieństwo, że jedna zmienna może być użytecznym predyktorem drugiej zmiennej. Jednak warto pamiętać, że korelacja nie oznacza przyczynowości, więc należy podejść ostrożnie do wnioskowania na temat predykcji na podstawie korelacji między zmiennymi.